In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import os

# Path to the folder containing CSV files
folder_path = '/content/drive/My Drive/Preprocessed'

# Combine all CSV files into one DataFrame
all_data = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Ensure it's a CSV file
        file_path = os.path.join(folder_path, file_name)
        # Specify the semicolon as the delimiter
        temp_data = pd.read_csv(file_path, sep=';')
        all_data = pd.concat([all_data, temp_data], ignore_index=True)

# Display the combined dataset
print("Combined DataFrame:")
print(all_data.info())
print(all_data.head())


In [ ]:
import pandas as pd

# Load one of the CSV files for inspection
file_path = '/content/drive/My Drive/Preprocessed/HUPA0015P.csv'  # Replace with the actual path
data = pd.read_csv(file_path, sep=';', header=None)  # Use sep=';' to handle semicolon delimiter

# Display the first few rows
print(data.head())

In [ ]:
data = pd.read_csv(file_path, sep=';', header=0, engine='python')  # Use 'python' engine for better compatibility
print(data.columns)
print(data.head())


In [ ]:
import os

# Path to the folder
folder_path = '/content/drive/My Drive/Preprocessed'

all_data = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)

        # Read and split the data
        temp_data = pd.read_csv(file_path, header=None)
        temp_data_split = temp_data[0].str.split(';', expand=True)
        temp_data_split.columns = ['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']

        # Combine into the main DataFrame
        all_data = pd.concat([all_data, temp_data_split], ignore_index=True)

print(all_data.info())
print(all_data.head())


In [ ]:

import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Combine all preprocessed CSV files
preprocessed_dir = '/content/drive/My Drive/Preprocessed'
all_data = pd.DataFrame()

for file_name in os.listdir(preprocessed_dir):
    file_path = os.path.join(preprocessed_dir, file_name)
    # Read and split the data, assuming semicolon delimiter and no header
    temp_data = pd.read_csv(file_path, header=None)
    temp_data_split = temp_data[0].str.split(';', expand=True)
    temp_data_split.columns = ['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']
    all_data = pd.concat([all_data, temp_data_split], ignore_index=True)
# Handle missing values
all_data = all_data.dropna()

# Separate features and target
target_column = 'glucose'  # Replace with your actual target column name
X = all_data.drop(columns=[target_column, 'time'])  # Drop 'time' column from features
y = all_data[target_column]

# Convert columns to numeric, errors='coerce' to handle potential non-numeric values
X = X.apply(pd.to_numeric, errors='coerce')

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Define the MLP model
mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
mlp_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
mlp_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Make predictions on the test set
predictions_mlp = mlp_model.predict(X_test)

# Define thresholds for categorization
thresholds = [70, 140]  # Example thresholds

# Categorize predictions and true values
y_test_cat = np.digitize(y_test, thresholds)
predictions_mlp_cat = np.digitize(predictions_mlp, thresholds)

# Calculate confusion matrix
cm_mlp = confusion_matrix(y_test_cat, predictions_mlp_cat)

# Visualize confusion matrix
sns.heatmap(cm_mlp, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Calculate metrics
precision_mlp = precision_score(y_test_cat, predictions_mlp_cat, average='weighted')
recall_mlp = recall_score(y_test_cat, predictions_mlp_cat, average='weighted')
accuracy_mlp = accuracy_score(y_test_cat, predictions_mlp_cat)
f1_mlp = f1_score(y_test_cat, predictions_mlp_cat, average='weighted')

# Print metrics
print("MLP Metrics:")
print("Precision:", precision_mlp)
print("Recall:", recall_mlp)
print("Accuracy:", accuracy_mlp)
print("F1-score:", f1_mlp)